# LIBS

In [33]:
import cv2
import matplotlib.pyplot as plt
from matplotlib import axes
import numpy as np
from PIL import Image
from scipy.ndimage import gaussian_filter, sobel, filters
import argparse
import imutils
import os
import sys

# FIND CONTOURS

In [34]:
def detect(frame):
    
    px_cm_x = 640/0.24 #Matrix size
    px_cm_y = 480/0.18 #Matrix size
    focal_length_cm = 0.42 #Camera focal length
    px_size = 0.000119 
    kernel = np.ones((5,5),np.uint8)
    kernelDilation = np.ones((3,3),np.uint8)
    #for file in os.listdir('Test/'):
    #file = 'box1.jpg'
    #print (file)
    table = 52 # Camera - floor
    box_h = 5 # Camera - box
    dist_box_cm = table - box_h
    #print( px_cm_x, px_cm_y)
    image = frame
    #Resize
    scale_percent = 60 # percent of original size
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized = cv2.resize(image, dim)
    
    #HSV Test
    hsv = cv2.cvtColor(resized, cv2.COLOR_BGR2HSV)
    
    s_uint8 = hsv[:,:,1]
    s_int32= np.int32(s_uint8) - 70
    s_uint8 = np.uint8(s_int32.clip(min=0))
    hsv[:,:,1] = 128
    
    v_uint8 = hsv[:,:,2]
    v_int32= np.int32(v_uint8) - 128
    v_uint8 = np.uint8(v_int32.clip(min=0))
    hsv[:,:,2] = 128
    
    hsv2bgr =  cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    ###
    ratio = resized.shape[0] / float(resized.shape[0])
    #ratio = hsv2bgr.shape[0] / float(hsv2bgr.shape[0])
    #Grayscale
    gray = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)
    #gray = cv2.cvtColor(hsv2bgr, cv2.COLOR_BGR2GRAY)
    #Blur
    #blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    blurred = cv2.bilateralFilter(gray,6,75,75)
    #Canny
    edges = cv2.Canny(blurred,0,80)
    #Thresh
    closing = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
    thresh = cv2.dilate(closing,kernelDilation,iterations = 6)
    #thresh  = cv2.adaptiveThreshold(closing,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
    #            cv2.THRESH_BINARY,7,2)
    try:
        cnts = cv2.findContours(thresh, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

        cnts = imutils.grab_contours(cnts)
        thresh = cv2.cvtColor(thresh,cv2.COLOR_GRAY2RGB)
        rectAreas = list()
        rectShapes = list()
        for c in cnts:
            M = cv2.moments(c)
            c = c.astype("float")
            c *= ratio
            c = c.astype("int")
            rectAreas.append(cv2.contourArea(c))
            rectShapes.append(c)
        #cv2.drawContours(resized, cnts, -1, (0, 255, 0), 2)
        #maxIndex = rectAreas.index(max(rectAreas))
        #del rectAreas[maxIndex]
        #del rectShapes[maxIndex]
        maxIndex = rectAreas.index(max(rectAreas))

        hull = cv2.convexHull(rectShapes[maxIndex], returnPoints=True)
        
        box = cv2.approxPolyDP(hull, 0.09 * cv2.arcLength(hull,True), True)
        
        
        #cv2.polylines(resized, [box],True,(0,0,255),4)
        cv2.drawContours(resized, cnts, -1, (0, 255, 0), 2)
        if len(box) == 4:
            cv2.polylines(resized, [hull],True,(0,0,255),2)
            #cv2.polylines(resized, [box],True,(255,0,0),4)
            #print(box)
            rect = cv2.minAreaRect(box)
            box1 = cv2.boxPoints(rect) # cv2.boxPoints(rect) for OpenCV 3.x
            box1 = np.int0(box1)
            cv2.drawContours(resized,[box1],0,(0,255,0),2)
            print(box1)
            #print(box)
            cv2.putText(resized, f'{box1[0]}', (box1[0][0],box1[0][1]), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,0,255), 1,1)
            cv2.putText(resized, f'{box1[1]}', (box1[1][0],box1[1][1]), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,0,255), 1,1)
            cv2.putText(resized, f'{box1[2]}', (box1[2][0],box1[2][1]), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,0,255), 1,1)
            cv2.putText(resized, f'{box1[3]}', (box1[3][0],box1[3][1]), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,0,255), 1,1)
            px_len1 = abs(np.sqrt((box1[0][0]- box1[1][0])**2+(box1[0][1]- box1[1][1])**2))
            px_len2 = abs(np.sqrt((box1[1][0]- box1[2][0])**2+(box1[1][1]- box1[2][1])**2))
            print('px_len:',px_len1,px_len2 )
            size_x_cm_mtx = px_len1/px_cm_x
            size_y_cm_mtx = px_len2/px_cm_y
            H1 = (((dist_box_cm*size_x_cm_mtx)-(focal_length_cm*size_x_cm_mtx))/focal_length_cm)/(scale_percent/100)
            H2 = ((dist_box_cm*size_y_cm_mtx-focal_length_cm*size_y_cm_mtx)/focal_length_cm)/(scale_percent/100)
            print(H1, H2)
            #cv2.putText(resized, f'{H1}, {H2}', (box[3][0][0],box[3][0][1]+20), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255,0,255), 1,1) 
            
            
            
            
        else:
            pass
            #print("Error")
        cv2.imwrite(f'Result/{file}',resized)
        cv2.imwrite(f'Result/1{file}',thresh)
        new_dims =(image.shape[0], image.shape[1])
        #resized = cv2.resize(resized, new_dims)
        return resized, thresh, blurred
    except:
        return resized, thresh, blurred

In [35]:
def in_range(img,lower, upper, show = False):
    lower = np.array(lower, dtype = "uint8")
    upper = np.array(upper, dtype = "uint8")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask =cv2.inRange(img, lower, upper)
    if show:
        cv2.imshow('mask', mask)
    return mask
    

In [36]:
def color_mask(frame):
    x = 310
    y = 0
    lower = [0,43,123]
    upper = [48,255,255]
    rectAreas = []
    rectShapes = []
    lower2 = [0,62,142]
    upper2 = [45,255,255]
    h,w = frame.shape[:2]
    img = frame.copy()
    mask = in_range(frame,lower2,upper2)
    #cv2.imshow('mask1', mask)
    mask2 = np.zeros(frame.shape,np.uint8)
    mask2[y:y+h,x:x+w] = frame[y:y+h,x:x+w]
    #cv2.imshow('mask2', mask2)
    mask2 = in_range(mask2,lower,upper)
    #cv2.imshow('mask3', mask2)
    mask = mask+mask2
    #cv2.imshow('mask4', mask)
    kernel = np.ones((5,5),np.uint8)
    mask = cv2.morphologyEx(mask,cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask,cv2.MORPH_CLOSE, kernel)
    mask = cv2.erode(mask, kernel, iterations = 3)
    mask = cv2.dilate(mask, kernel, iterations = 3)
    cv2.imshow('mask', mask)
    cnts = cv2.findContours(mask, cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

    cnts = imutils.grab_contours(cnts)
    #thresh = cv2.cvtColor(thresh,cv2.COLOR_GRAY2RGB)
    #rectAreas = list()
    #rectShapes = list()
    for c in cnts:
        #M = cv2.moments(c)
        c = c.astype("float")
        c *= ratio
        c = c.astype("int")
        rectAreas.append(cv2.contourArea(c))
        rectShapes.append(c)
    maxIndex = rectAreas.index(max(rectAreas))
    #cv2.drawContours(img, rectShapes[maxIndex], -1, (0, 255, 0), 2)
    rect = cv2.minAreaRect(rectShapes[maxIndex])
    box1 = cv2.boxPoints(rect) # cv2.boxPoints(rect) for OpenCV 3.x
    box1 = np.int0(box1)
    cv2.drawContours(img,[box1],0,(0,255,0),2)
    cv2.imshow('img', img)

In [42]:


#for file in os.listdir('Test/'):
file = 'box26.jpg'


print (file)
image = cv2.imread(f'Test/{file}')
color_mask(image)
detect(image)
cv2.waitKey()
cv2.destroyAllWindows()
#Resize
    

box26.jpg


NameError: name 'ratio' is not defined

# Trying linear ransac


In [23]:
def calc_model(points):
    k = (points[0][1] - points[1][1])/(points[0][0]-points[1][0]+ sys.float_info.epsilon)
    b = points[1][1]-k*points[1][0]
    return k, b

In [24]:
def intercept_dist(x0,y0,k,b,points):
    x = (x0+k*y0-k*b)/(1+k**2)
    y = (k*x0 + (k**2)*y0 - (k**2)*b)/(1 + k**2) + b
    f = (points[1][0]-points[0][0])*(y-points[0][1])-(points[1][1]-points[0][1])*(x-points[0][0])+ sys.float_info.epsilon
    f1 = np.divide(f,abs(f))    
    dist = np.sqrt((x-x0)**2+(y-y0)**2)
    dist_sgn = dist*f1
    #print(type(dist_sgn))
    return dist_sgn

In [25]:
def ransac_plot(x,y,k,b,thresh,pts, closepts, newx, newy):
    #plt.figure("Ransac", figsize=(5., 5.))
    plt.plot(newx,newy,marker='o', color='#ff00cc', linestyle='None', alpha=0.5)
    #plt.plot(x,y,marker='o', color='#0000FF', linestyle='None', alpha=0.2)
    #print(k*x+b) 
    plt.plot(x, k*x+b,color='#00cc00', linewidth=2)
    bias = thresh/(np.arctan(1/(k+sys.float_info.epsilon)))
    #print(bias)
    plt.plot(x, k*x+(b-bias),color='#ffff00', linewidth=0.5)
    plt.plot(x, k*x+(b+bias),color='#ffff00', linewidth=0.5)
    plt.plot(pts[:,0],pts[:,1],marker='o', color='#FF0000', linestyle='None', alpha=1)
    plt.plot(closepts[:,0],closepts[:,1],marker='o', color='#FFFF00', linestyle='None', alpha=0.4)
    #plt.plot(perfectx, perfecty,color='#ff00ff', linewidth=2)
    plt.ylim(200, 1000)
    plt.show()

In [26]:
def del_pts(delList, data):
    listToDel = []
    for i in delList:
        listToDel.append(np.where((data[:,0] == i[0]) & (data[:,1] == i[1]))[0][0])
    data1 = np.delete(data, listToDel,axis=0)
    return data1

In [27]:
np.random.seed(2)
data = rectShapes[maxIndex].reshape(rectShapes[maxIndex].shape[0],2)
k_iterations =200
n_inliners = 2
for i in range(4):
    best_inliers = None
    best_close_pts = None
    t_thresh =15
    d_close_pts = 0
    iterations = 0
    model_k = 0
    model_b = 0
    k = 0
    b = 0
    for iteration in range(k_iterations):
        #print('iteration',i)
        add_inliers_l = list()
        add_inliers_r = list()
        add_inliers = list()
        all_indices = np.arange(data.shape[0])
        np.random.shuffle(all_indices)
        indices_1 = all_indices[:n_inliners]
        indices_2 = all_indices[n_inliners:]
        maybeInLiers = data[indices_1,:]
        otherPoints = data[indices_2,:]
        k, b = calc_model(maybeInLiers)
        dist = intercept_dist(otherPoints[:,0],otherPoints[:,1],k,b,maybeInLiers)
        for i in dist:
            if abs(i) < t_thresh:
                add_inliers.append(otherPoints[np.where(dist == i)[0][0]])
                if i<0:
                    add_inliers_l.append(otherPoints[np.where(dist == i)[0][0]])
                elif i>0:
                    add_inliers_r.append(otherPoints[np.where(dist == i)[0][0]])
        if len(add_inliers)>d_close_pts:
            if len(add_inliers_l)/len(add_inliers)>0.1 and len(add_inliers_l)/len(add_inliers)<0.9:
                d_close_pts = len(add_inliers)
                model_k = k
                model_b = b
                best_inliers = maybeInLiers
                best_close_pts = np.concatenate((np.array(add_inliers),best_inliers)) 

                print('found better ', d_close_pts, model_k, model_b, 'at ',iteration, ' iteration')
    print( 'Best model: k = ',model_k,' b = ',model_b, 'close pts: ',d_close_pts)
    new_data = del_pts(best_close_pts, data)

    print (data.shape, new_data.shape)
    ransac_plot(data[:,0],data[:,1],model_k,model_b,t_thresh,best_inliers,best_close_pts,new_data[:,0],new_data[:,1]) 
    data = new_data


NameError: name 'rectShapes' is not defined

# RANSAC

In [38]:
def calc_model(points, E_max): #len(points) = 4 
    AB = np.array([[points[1][0]-points[0][0]],[points[1][1]-points[0][1]]]) 
    AD = np.array([[points[3][0]-points[0][0]],[points[3][1]-points[0][1]]]) 
    CB = np.array([[points[1][0]-points[2][0]],[points[1][1]-points[2][1]]]) 
    CD = np.array([[points[3][0]-points[2][0]],[points[3][1]-points[2][1]]]) 
    
    normABAD = abs(AB[0]*AD[0]+AB[1]*AD[1])
    normCBCD = abs(CB[0]*CD[0]+CB[1]*CD[1])
    
    parallADCB = abs((AD[0]+ sys.float_info.epsilon)/(CB[0]+ sys.float_info.epsilon) + (AD[1]+ sys.float_info.epsilon)/(CB[1]+ sys.float_info.epsilon))
    
    E = normABAD + normCBCD + parallADCB
    #print('normABAD',normABAD,'normCBCD',normCBCD,'parallADCB',parallADCB, 'E', E)
    if E < E_max:
        delim0 = points[0][0]-points[1][0]+sys.float_info.epsilon
        delim1 = points[0][0]-points[3][0]+sys.float_info.epsilon
        delim2 = points[2][0]-points[1][0]+sys.float_info.epsilon
        delim3 = points[2][0]-points[3][0]+sys.float_info.epsilon

        kAB = (points[0][1]-points[1][1])/delim0
        bAB = points[1][1]-kAB*points[1][0]

        kAD = (points[0][1]-points[3][1])/delim1
        bAD = points[3][1]-kAD*points[3][0]

        kCB = (points[2][1]-points[1][1])/delim2
        bCB = points[1][1]-kCB*points[1][0]

        kCD = (points[2][1]-points[3][1])/delim3
        bCD = points[3][1]-kCD*points[3][0]  
    else:
        kAB = 0
        kAD = 0
        kCB = 0
        kCD = 0
        bAB = 0
        bAD = 0
        bCB = 0
        bCD = 0
    params ={ 'kAB':kAB,'kAD':kAD,'kCB':kCB,'kCD':kCD,'bAB':bAB,'bAD':bAD,'bCB':bCB,'bCD':bCD}
    #print(params)
    return params

In [39]:
def intercept_dist(x0,y0,k,b,points):
    x = (x0+k*y0-k*b)/(1+k**2)
    y = (k*x0 + (k**2)*y0 - (k**2)*b)/(1 + k**2) + b
    f = (points[1][0]-points[0][0])*(y-points[0][1])-(points[1][1]-points[0][1])*(x-points[0][0])+ sys.float_info.epsilon
    f1 = np.divide(f,abs(f))    
    dist = np.sqrt((x-x0)**2+(y-y0)**2)
    dist_sgn = dist*f1
    print(type(dist_sgngn))
    return dist_sgn

In [40]:
def ransac_plot(x,y,params,thresh, pts):
    
    bias1 = thresh/(np.arctan(1/(params['kAB']+sys.float_info.epsilon)))
    #print(bias1.shape)
    bias2 = thresh/(np.arctan(1/(params['kAD']+sys.float_info.epsilon)))
    bias3 = thresh/(np.arctan(1/(params['kCB']+sys.float_info.epsilon)))
    bias4 = thresh/(np.arctan(1/(params['kCD']+sys.float_info.epsilon)))
    plt.plot(x,y,marker='o', color='#0000cc', linestyle='None', alpha=0.5)
    
    plt.plot(x, params['kAB']*x+params['bAB'],color='#00cc00', linewidth=2)
    plt.plot(x, params['kAB']*x+(params['bAB']+bias1),color='#ffff00', linewidth=0.5)
    plt.plot(x, params['kAB']*x+(params['bAB']-bias1),color='#ffff00', linewidth=0.5)
    
    plt.plot(x, params['kAD']*x+params['bAD'],color='#cc0000', linewidth=2)
    plt.plot(x, params['kAD']*x+(params['bAD']+bias2),color='#ffff00', linewidth=0.5)
    plt.plot(x, params['kAD']*x+(params['bAD']-bias2),color='#ffff00', linewidth=0.5)
    
    plt.plot(x, params['kCB']*x+params['bCB'],color='#0000cc', linewidth=2)
    plt.plot(x, params['kCB']*x+(params['bCB']+bias3),color='#ffff00', linewidth=0.5)
    plt.plot(x, params['kCB']*x+(params['bCB']-bias3),color='#ffff00', linewidth=0.5)
    
    plt.plot(x, params['kCD']*x+params['bCD'],color='#00cccc', linewidth=2)
    plt.plot(x, params['kCD']*x+(params['bCD']+bias4),color='#ffff00', linewidth=0.5)
    plt.plot(x, params['kCD']*x+(params['bCD']-bias4),color='#ffff00', linewidth=0.5)
    
    plt.plot(pts[:,0],pts[:,1],marker='o', color='#FF0000', linestyle='None', alpha=1)
    plt.ylim(200, 1000)
    plt.show()

In [41]:
#np.random.seed(2)
data = rectShapes[maxIndex].reshape(rectShapes[maxIndex].shape[0],2)
E_max = 100
k_iterations =200
n_inliers = 4
t_thresh =10
d_close_pts = 0
iterations = 0
model_k = 0
model_b = 0
model_params = {'kAB':0,'kAD':0,'kCB':0,'kCD':0,'bAB':0,'bAD':0,'bCB':0,'bCD':0}
params = {}
for iteration in range(k_iterations):
    #print('iteration',i)
    add_inliers_l = list()
    add_inliers_r = list()
    add_inliers = list()
    all_indices = np.arange(data.shape[0])
    np.random.shuffle(all_indices)
    indices_1 = all_indices[:n_inliers]
    indices_2 = all_indices[n_inliers:]
    maybeInLiers = data[indices_1,:]
    otherPoints = data[indices_2,:]
    params = calc_model(maybeInLiers, E_max)
    dist0 = intercept_dist(otherPoints[:,0],otherPoints[:,1],params['kAB'],params['bAB'],maybeInLiers)
    dist1 = intercept_dist(otherPoints[:,0],otherPoints[:,1],params['kAD'],params['bAD'],maybeInLiers)
    dist2 = intercept_dist(otherPoints[:,0],otherPoints[:,1],params['kCB'],params['bCB'],maybeInLiers)
    dist3 = intercept_dist(otherPoints[:,0],otherPoints[:,1],params['kCD'],params['bCD'],maybeInLiers)
    #dist = np.concatenate((dist0, dist1,dist2,dist3))
    #print(dist)
    for i in dist0:
        if abs(i) < t_thresh:
            #print(otherPoints.shape)
            add_inliers.append(otherPoints[np.where(dist0 == i)[0][0]])
            if i<0:
                add_inliers_l.append(otherPoints[np.where(dist0 == i)[0][0]])
            elif i>0:
                add_inliers_r.append(otherPoints[np.where(dist0 == i)[0][0]])
    for i in dist1:
        if abs(i) < t_thresh:
            #print(otherPoints.shape)
            add_inliers.append(otherPoints[np.where(dist1 == i)[0][0]])
            if i<0:
                add_inliers_l.append(otherPoints[np.where(dist1 == i)[0][0]])
            elif i>0:
                add_inliers_r.append(otherPoints[np.where(dist1 == i)[0][0]])
    for i in dist2:
        if abs(i) < t_thresh:
            #print(otherPoints.shape)
            add_inliers.append(otherPoints[np.where(dist2 == i)[0][0]])
            if i<0:
                add_inliers_l.append(otherPoints[np.where(dist2 == i)[0][0]])
            elif i>0:
                add_inliers_r.append(otherPoints[np.where(dist2 == i)[0][0]])
    for i in dist3:
        if abs(i) < t_thresh:
            #print(otherPoints.shape)
            add_inliers.append(otherPoints[np.where(dist3 == i)[0][0]])
            if i<0:
                add_inliers_l.append(otherPoints[np.where(dist3 == i)[0][0]])
            elif i>0:
                add_inliers_r.append(otherPoints[np.where(dist3 == i)[0][0]])
    if len(add_inliers)>d_close_pts:
        if len(add_inliers_l)/len(add_inliers)>0.3 and len(add_inliers_l)/len(add_inliers)<0.7:
            d_close_pts = len(add_inliers)
            model_params = params
            best_inliers = maybeInLiers
            print('found better ', d_close_pts, model_params, 'at ',iteration, ' iteration')
    #ransac_plot(data[:,0],data[:,1],params,t_thresh,maybeInLiers)
print( 'Best model: ',model_params)
ransac_plot(data[:,0],data[:,1],model_params,t_thresh,best_inliers) 
    

NameError: name 'rectShapes' is not defined

# HSV Test

In [4]:
file = 'box19.jpg'
image = cv2.imread(f'Test/{file}')
scale_percent = 10 # percent of original size
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dim = (width, height)
resized = cv2.resize(image, dim)
#reshape = 
hsv = cv2.cvtColor(resized, cv2.COLOR_BGR2HSV)
# define range of blue color in HSV
'''
s_uint8 = hsv[:,:,1]
s_int32= np.int32(s_uint8) - 0
s_uint8 = np.uint8(s_int32.clip(min=0))
hsv[:,:,1] = s_uint8
'''
v_uint8 = hsv[:,:,2]
v_int32= np.int32(v_uint8) - 70
v_uint8 = np.uint8(v_int32.clip(min=0))
hsv[:,:,2] = 128

hsv2bgr =  cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
gray = cv2.cvtColor(hsv2bgr, cv2.COLOR_BGR2GRAY)
blurred = cv2.bilateralFilter(gray,6,75,75)
edges = cv2.Canny(blurred,0,70)

cv2.imshow('HSV', hsv)
cv2.imshow('RGB', resized)
cv2.imshow('hsv2bgr', hsv2bgr)
cv2.imshow('edges', edges)
cv2.waitKey()


-1

In [188]:
hsv[:,:,2]

array([[154, 154, 153, ...,  96,  92,  89],
       [154, 153, 154, ...,  96,  91,  89],
       [151, 154, 151, ...,  93,  91,  93],
       ...,
       [ 48,  49,  48, ...,  14,  16,  15],
       [ 50,  48,  46, ...,  16,  18,  12],
       [ 46,  48,  45, ...,  13,  11,  15]], dtype=uint8)

In [182]:
v_uint8 = hsv[:,:,2]
v_int32= np.int32(v_uint8) - 20
v_uint8 = np.uint8(v_int32.clip(min=0))


In [22]:
file = 'box13.jpg'
image = cv2.imread(f'Test/{file}')
scale_percent = 20 # percent of original size
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dim = (width, height)
resized = cv2.resize(image, dim)
#reshape = 
hsv = cv2.cvtColor(resized, cv2.COLOR_BGR2HSV)
hsv2bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
cv2.imshow('HSV', hsv)
cv2.imshow('RGB', resized)
cv2.imshow('hsv2bgr', hsv2bgr)
cv2.waitKey()

-1

In [24]:
resized

array([[[197, 203, 178],
        [195, 202, 181],
        [184, 193, 172],
        ...,
        [160, 171, 145],
        [155, 170, 139],
        [161, 178, 145]],

       [[194, 205, 173],
        [187, 199, 171],
        [190, 202, 174],
        ...,
        [152, 162, 139],
        [163, 176, 150],
        [157, 170, 144]],

       [[185, 199, 171],
        [185, 198, 172],
        [192, 202, 179],
        ...,
        [153, 161, 138],
        [164, 177, 151],
        [150, 160, 137]],

       ...,

       [[103, 111,  88],
        [121, 135, 111],
        [102, 118,  94],
        ...,
        [ 82,  92,  76],
        [ 75,  85,  72],
        [ 88,  95,  80]],

       [[115, 125, 108],
        [122, 133, 113],
        [ 88, 102,  78],
        ...,
        [ 96, 100,  88],
        [ 81,  88,  75],
        [ 80,  86,  75]],

       [[ 94, 115,  90],
        [111, 129, 100],
        [ 95, 115,  80],
        ...,
        [ 78,  86,  75],
        [ 74,  79,  70],
        [ 75,  79,  74]]

In [25]:
hsv

array([[[83, 31, 31],
        [80, 27, 27],
        [77, 28, 28],
        ...,
        [77, 39, 39],
        [75, 47, 47],
        [75, 47, 47]],

       [[80, 40, 40],
        [77, 36, 36],
        [77, 35, 35],
        ...,
        [77, 36, 36],
        [75, 38, 38],
        [75, 39, 39]],

       [[75, 36, 36],
        [75, 33, 33],
        [77, 29, 29],
        ...,
        [80, 36, 36],
        [75, 37, 37],
        [77, 37, 37]],

       ...,

       [[80, 53, 53],
        [73, 45, 45],
        [70, 52, 52],
        ...,
        [71, 44, 44],
        [67, 39, 39],
        [76, 40, 40]],

       [[72, 35, 35],
        [74, 38, 38],
        [73, 60, 60],
        ...,
        [80, 31, 31],
        [74, 38, 38],
        [74, 33, 33]],

       [[65, 55, 55],
        [71, 57, 57],
        [73, 78, 78],
        ...,
        [68, 33, 33],
        [73, 29, 29],
        [66, 16, 16]]], dtype=uint8)

In [26]:
hsv2bgr

array([[[30, 31, 27],
        [26, 27, 24],
        [26, 28, 24],
        ...,
        [36, 39, 33],
        [43, 47, 38],
        [43, 47, 38]],

       [[37, 40, 33],
        [33, 36, 30],
        [32, 35, 30],
        ...,
        [34, 36, 31],
        [35, 38, 32],
        [36, 39, 33]],

       [[33, 36, 30],
        [30, 33, 28],
        [27, 29, 25],
        ...,
        [34, 36, 31],
        [34, 37, 32],
        [35, 37, 32]],

       ...,

       [[49, 53, 41],
        [40, 45, 37],
        [44, 52, 41],
        ...,
        [39, 44, 36],
        [34, 39, 33],
        [37, 40, 34]],

       [[32, 35, 30],
        [34, 38, 32],
        [52, 60, 45],
        ...,
        [30, 31, 27],
        [35, 38, 32],
        [31, 33, 29]],

       [[45, 55, 43],
        [48, 57, 44],
        [64, 78, 54],
        ...,
        [30, 33, 29],
        [27, 29, 26],
        [15, 16, 15]]], dtype=uint8)

In [75]:
dim

(576, 432)

In [77]:
new_dims =(image.shape[0], image.shape[1])


In [78]:
new_dims

(1440, 1920)